##### 问题:
我们正在编写使用回调函数的代码，但是担心小型函数在代码中大肆泛滥，程序的控
制流会因此而失控。我们希望能有某种方法使代码看起来更像一般的过程式步骤。


##### 解决方案:
我们可以通过生成器和协程将回调函数内联到一个函数中。为了说明，假设有一个函
数会按照下面的方式调用回调函数（参见 7.10 节）：

In [5]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    # Invoke the callback with the result
    callback(result) 


现在看看接下来的支持代码，这里涉及一个 Async 类和 inlined_async 装饰器：

In [6]:
from queue import Queue
from functools import wraps
class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args
def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)  
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

这两段代码允许我们通过 yield 语句将回调函数变为内联式的，示例如下:

In [7]:
def add(x, y):
    return x + y
@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

In [8]:
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


除了那个特殊的装饰器和对 yield 的使用之外，我们会发现代码中根本就没有出现回调
函数（它们只是隐藏在幕后了）。

本节将真正考验一下读者对回调函数、生成器以及程序控制流方面的掌控情况。

首先，在涉及回调函数的代码中，问题的关键就在于当前的计算会被挂起，然后在稍
后某个时刻再得到恢复。当计算得到恢复时，回调函数将得以继续处理执行。示例中
的 apply_async()函数对执行回调函数的关键部分做了简单的说明，尽管在现实世界中这
会复杂得多（涉及线程、进程、事件处理例程等）。

将计算挂起之后再恢复，这个思想非常自然地同生成器函数对应了起来。具体来说就
是 yield 操作使得生成器函数产生出一个值然后就挂起，后续调用生成器的__next__()
或者 send()方法会使得它再次启动。

鉴于此，本节的核心就在 inline_async()装饰器函数中。关键点就是对于生成器函数的
所有 yield 语句装饰器都会逐条进行跟踪，一次一个。为了做到这点，我们创建了一
个队列用来保存结果，初始时用 None 来填充。之后通过循环将结果从队列中取出，
然后发送给生成器，这样就会产生下一次 yield，此时就会接收到 Async 的实例。然
后在循环中查找函数和参数，开始异步计算 apply_async()。但是，这个过程中最为隐
蔽的部分就在于这里没有使用普通的回调函数，回调过程被设定到队列的 put()方法
中了。

此时应该可以精确描述到底都发生了些什么。主循环会迅速回到顶层，并在队列中执
行一个 get()操作。如果有数据存在，那它就一定是由 put()回调产生的结果。如果什么
都没有，操作就会阻塞，等待之后某个时刻会有结果到来。至于结果要如何产生，这取
决于 apply_async()函数的实现。
